In [1]:
import numpy as np
from PIL import Image
import trimesh
import pyrender

In [2]:
correct_glb_p = "/home/quanta/Downloads/5d81bcc38969431e9d18764e2c72d285.glb"

render_glb_1_p = '/home/quanta/Downloads/render_error/001af60e5741419cb7a5767e17c270b9.glb'
render_glb_2_p = '/home/quanta/Downloads/render_error/09af633b3e2d4fd0add9918da29fa668.glb'


points_glb_1_p = '/home/quanta/Downloads/points_error/4f7fb039a1ac4cd988f7b57bad26bdcf.glb'
points_glb_2_p = '/home/quanta/Downloads/points_error/fc239e705c194ec580aa5231a0f32377.glb'

In [3]:
import os, sys
sys.path.append("..")

from src.utils.data_utils import normalize_mesh

/home/quanta/.conda/envs/partcrafter/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
mesh = trimesh.load(render_glb_2_p)
mesh = normalize_mesh(mesh).to_geometry()

In [5]:
ALLOWED_MODES = {"L", "RGB", "RGBA", "F"}

def normalize_texture_pil(im: Image.Image):
    if not isinstance(im, Image.Image):
        return im
    m = im.mode
    if m in ALLOWED_MODES:
        return im
    if m == "1":  # 1bit -> 8bit 灰度
        return im.convert("L")
    if m == "P":  # 调色板 -> RGBA
        return im.convert("RGBA")
    if m == "LA":  # 灰度+alpha -> RGBA
        return im.convert("RGBA")
    if m in {"CMYK", "YCbCr", "HSV"}:  # 这些已是 uint8，但通道不兼容 -> RGB
        return im.convert("RGB")
    if m in {"I;16", "I;16B", "I;16L", "I"}:  # 16/32 位整型 -> 8bit 灰度
        arr = np.array(im, dtype=np.uint32)
        arr8 = (arr >> 8).astype(np.uint8)  # 不做拉伸，仅丢高 8 位
        return Image.fromarray(arr8, mode="L")
    # 兜底
    return im.convert("RGB")


def coerce_trimesh_textures_pil(tm):
    vis = getattr(tm, "visual", None)
    if vis is None:
        return
    mats = []
    if getattr(vis, "material", None) is not None:
        mats.append(vis.material)
    if getattr(vis, "materials", None):
        mats.extend(list(vis.materials))

    fields = (
        "image",  # SimpleMaterial
        "baseColorTexture",
        "metallicRoughnessTexture",
        "normalTexture",
        "occlusionTexture",
        "emissiveTexture",
        "diffuseTexture",
        "specularGlossinessTexture",
    )
    for m in mats:
        for f in fields:
            if hasattr(m, f):
                tex = getattr(m, f)
                setattr(m, f, normalize_texture_pil(tex))

In [6]:
pyrender.Scene.from_trimesh_scene(trimesh.Scene(coerce_trimesh_textures_pil(mesh)))

In [7]:
# pyrender.Scene.from_trimesh_scene(trimesh.Scene(mesh))

In [8]:
mesh = trimesh.load(points_glb_1_p, process=False)
mesh = normalize_mesh(mesh)

In [9]:
def filter_mesh(obj):
    import trimesh
    if isinstance(obj, trimesh.Trimesh):
        return obj.copy()
    if isinstance(obj, trimesh.Scene):
        sc = trimesh.Scene()
        for name, g in obj.geometry.items():
            if isinstance(g, trimesh.Trimesh):
                sc.add_geometry(g.copy(), node_name=name)
        return sc
    return trimesh.Scene()


In [10]:
mesh.geometry

OrderedDict([('Object_0',
              <trimesh.Trimesh(vertices.shape=(10326, 3), faces.shape=(19616, 3))>),
             ('Object_1',
              <trimesh.Trimesh(vertices.shape=(481, 3), faces.shape=(549, 3))>),
             ('Object_2', <trimesh.PointCloud(vertices.shape=(1, 3))>),
             ('Object_3',
              <trimesh.Trimesh(vertices.shape=(51811, 3), faces.shape=(99956, 3))>)])

In [11]:
filter_mesh(mesh).geometry, mesh.geometry

(OrderedDict([('geometry_0',
               <trimesh.Trimesh(vertices.shape=(10326, 3), faces.shape=(19616, 3))>),
              ('geometry_1',
               <trimesh.Trimesh(vertices.shape=(481, 3), faces.shape=(549, 3))>),
              ('geometry_2',
               <trimesh.Trimesh(vertices.shape=(51811, 3), faces.shape=(99956, 3))>)]),
 OrderedDict([('Object_0',
               <trimesh.Trimesh(vertices.shape=(10326, 3), faces.shape=(19616, 3))>),
              ('Object_1',
               <trimesh.Trimesh(vertices.shape=(481, 3), faces.shape=(549, 3))>),
              ('Object_2', <trimesh.PointCloud(vertices.shape=(1, 3))>),
              ('Object_3',
               <trimesh.Trimesh(vertices.shape=(51811, 3), faces.shape=(99956, 3))>)]))

In [12]:
from src.utils.data_utils import scene_to_parts, mesh_to_surface, normalize_mesh

In [13]:
scene_to_parts(filter_mesh(mesh), return_type="point", normalize=False)

[{'surface_points': TrackedArray([[-1.00040272e+03, -3.77285787e-01, -3.10233215e+02],
                [-1.25877906e+03,  7.84497425e+01, -5.64346531e+02],
                [-1.35625991e+03, -7.16874190e+01, -1.79921706e+02],
                ...,
                [-1.09764060e+03,  1.91336005e+01, -3.11498011e+02],
                [-1.49662780e+03,  8.64989612e+01, -2.84121054e+02],
                [-1.55877873e+03,  1.17092021e+01, -3.53887548e+02]]),
  'surface_normals': array([[ 0.6708733 , -0.60269129,  0.43207895],
         [ 0.07963062,  0.98761243, -0.13520596],
         [-0.38313995, -0.69707845,  0.60604077],
         ...,
         [-0.72697954, -0.33263551, -0.60071155],
         [-0.55598125,  0.56689484, -0.60787753],
         [ 0.75852426, -0.41799061, -0.4999248 ]])},
 {'surface_points': TrackedArray([[-1310.66391525,   -92.75666879,  -223.47672123],
                [-1309.43851169,  -118.87162696,  -581.16022797],
                [-1308.70075597,  -121.26718355,  -417.7588